# Lab 08 - Redshift
#### Izzy Valdivia
#### 11/22/2025


### Native Optimized CTAS Table:  
`optimized_orders`  
This represents the data that gets loaded into Redshift that gets queries more frequently
 
______________
  
### External Table:  
`spectrum_raw.orders`  
Not in redshift. This was used to load data into redshift, though. 


#### Goal: 
Analyze the tradeoff between query performance and query flexibility when using an optimized CTAS table versus an external table.

In [12]:
import boto3
import time
import json


In [14]:
# Establish connection to redshift 
session = boto3.Session(region_name='us-west-2', profile_name='default')
client = session.client('redshift-data')

# Function to execute the sql query!
def execute_and_wait(sql, cluster_id, database, db_user):
    """Execute SQL and wait for completion, return statement metadata"""
    response = client.execute_statement(
        ClusterIdentifier=cluster_id,
        Database=database,
        DbUser=db_user,
        Sql=sql
    )
    stmt_id = response['Id']

    # Poll until complete
    while True:
        status = client.describe_statement(Id=stmt_id)
        if status['Status'] in ['FINISHED', 'FAILED', 'ABORTED']:
            break
        time.sleep(1)

    return status  # Contains Duration, ResultRows, etc.


In [17]:
query_sql = """
SELECT
    category,
    region,
    COUNT(*) as order_count,
    SUM(extended_price) as total_revenue,
    AVG(discount_rate) as avg_discount,
    COUNT(DISTINCT product_sku) as unique_products
FROM optimized_orders
WHERE ts >= '2024-06-01' AND ts < '2024-07-01'
    AND category IN ('analytics', 'compute', 'observability')
    AND region IN ('us-east', 'us-west', 'eu-west')
GROUP BY category, region
ORDER BY total_revenue DESC;"""

explain_result = execute_and_wait(
    f"EXPLAIN {query_sql}",
    # grab the below values from your stack output
    cluster_id='wk08-redshift-cluster',
    database='dev',
    db_user='rsadmin'
)


In [18]:
print(explain_result)

{'ClusterIdentifier': 'wk08-redshift-cluster', 'CreatedAt': datetime.datetime(2025, 11, 23, 11, 19, 44, 377000, tzinfo=tzlocal()), 'Database': 'dev', 'DbUser': 'rsadmin', 'Duration': 33790564, 'HasResultSet': True, 'Id': '24ef45b9-56ce-4e8a-9751-d2a4efa8a1a9', 'QueryString': "EXPLAIN \nSELECT\n    category,\n    region,\n    COUNT(*) as order_count,\n    SUM(extended_price) as total_revenue,\n    AVG(discount_rate) as avg_discount,\n    COUNT(DISTINCT product_sku) as unique_products\nFROM optimized_orders\nWHERE ts >= '2024-06-01' AND ts < '2024-07-01'\n    AND category IN ('analytics', 'compute', 'observability')\n    AND region IN ('us-east', 'us-west', 'eu-west')\nGROUP BY category, region\nORDER BY total_revenue DESC;", 'RedshiftPid': 1073987723, 'RedshiftQueryId': -1, 'ResultFormat': 'json', 'ResultRows': 9, 'ResultSize': 906, 'Status': 'FINISHED', 'UpdatedAt': datetime.datetime(2025, 11, 23, 11, 19, 44, 938000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '4b41811a-b387-4